In [106]:
import urllib, string, os, math, cmath, re, nltk, gensim, time, sklearn, matplotlib 
import numpy as np
import pandas as pd
import _pickle as cPickle
from time import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
%matplotlib inline

from nltk.corpus import stopwords
from nltk.corpus import stopwords
from gensim.models import LdaModel
from gensim import models, corpora, similarities
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer 
from nltk import FreqDist
from scipy.stats import entropy

## Load data/corpus and model

In [107]:
# load arxiv data with cleaned abstracts
data_df = pd.read_csv("cleaned_data.csv");
data_df = data_df.drop(columns=["Unnamed: 0"])

In [125]:
from gensim import corpora
import pickle

# load corpus and dictionary 
corpus = pickle.load(open("corpus.pkl","rb"))
dictionary = pickle.load(open("dictionary.pkl","rb"))

In [126]:
data_df.head(3)

,id,authors,published,title,abstitle,prim_cat,word_count,title_count,cleaned_abstitle
0,cs/9308101,M. L. Ginsberg,"Journal of Artificial Intelligence Research, V...",dynamic backtracking,dynamic backtracking occasional need return sh...,cs.AI,445,20,dynam backtrack occasion need return shallow p...
1,cs/9308102,M. P. Wellman,"Journal of Artificial Intelligence Research, V...",a marketoriented programming environment and i...,marketoriented programming environment applica...,cs.AI,798,106,marketori program environ applic distribut mul...
2,cs/9309101,"I. P. Gent, T. Walsh","Journal of Artificial Intelligence Research, V...",an empirical analysis of search in gsat,empirical analysis search gsat describe extens...,cs.AI,829,39,empir analysi search gsat describ extens studi...


In [127]:
print(dictionary)

Dictionary(81701 unique tokens: ['approach', 'avoid', 'backtrack', 'complet', 'control']...)


In [111]:
print(len(data_df),data_df['id'].duplicated().sum())

49207 0


In [112]:
# Dropping duplicates!
print("duplicated entries = ", data_df['id'].duplicated().sum())
data_df = data_df.drop_duplicates(subset="id")
data_df = data_df.reset_index(drop=True)
print("duplicated entries after removal = ", data_df['id'].duplicated().sum())

duplicated entries =  0
duplicated entries after removal =  0


## Loading LDA model

In [113]:
# loading the model
ldamodel = LdaModel.load('model14.gensim')

# checking the topics
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.030*"gener" + 0.028*"imag" + 0.018*"network" + 0.013*"use"')
(1, '0.014*"social" + 0.014*"use" + 0.010*"model" + 0.008*"emot"')
(2, '0.024*"data" + 0.022*"method" + 0.021*"learn" + 0.017*"featur"')
(3, '0.030*"problem" + 0.019*"optim" + 0.012*"search" + 0.012*"cluster"')
(4, '0.040*"user" + 0.022*"recommend" + 0.020*"data" + 0.015*"learn"')
(5, '0.014*"research" + 0.014*"machin" + 0.013*"learn" + 0.013*"human"')
(6, '0.015*"function" + 0.015*"gradient" + 0.015*"optim" + 0.013*"bound"')
(7, '0.025*"data" + 0.019*"use" + 0.018*"predict" + 0.017*"detect"')
(8, '0.012*"logic" + 0.012*"program" + 0.011*"use" + 0.009*"set"')
(9, '0.038*"learn" + 0.021*"agent" + 0.020*"polici" + 0.019*"reinforc"')
(10, '0.021*"languag" + 0.018*"model" + 0.017*"task" + 0.012*"text"')
(11, '0.030*"learn" + 0.024*"train" + 0.018*"data" + 0.016*"method"')
(12, '0.061*"network" + 0.036*"neural" + 0.022*"deep" + 0.017*"train"')
(13, '0.052*"graph" + 0.035*"network" + 0.035*"adversari" + 0.029*"attack"')


## Getting the topic distribution of each document

In [114]:
print(help(ldamodel.get_document_topics))
print(ldamodel.get_document_topics(corpus[0]))
print(ldamodel.get_document_topics(corpus[1]))
print(ldamodel.get_document_topics(corpus[2]))

Help on method get_document_topics in module gensim.models.ldamodel:

get_document_topics(bow, minimum_probability=None, minimum_phi_value=None, per_word_topics=False) method of gensim.models.ldamulticore.LdaMulticore instance
    Get the topic distribution for the given document.
    
    Parameters
    ----------
    bow : corpus : list of (int, float)
        The document in BOW format.
    minimum_probability : float
        Topics with an assigned probability lower than this threshold will be discarded.
    minimum_phi_value : float
        If `per_word_topics` is True, this represents a lower bound on the term probabilities that are included.
         If set to None, a value of 1e-8 is used to prevent 0s.
    per_word_topics : bool
        If True, this function will also return two extra lists as explained in the "Returns" section.
    
    Returns
    -------
    list of (int, float)
        Topic distribution for the whole document. Each element in the list is a pair of a topi

In [128]:
def topic_output(model,x_bow,n_topics=14):
    out=[0]*n_topics;
    topics = model.get_document_topics(x_bow); 
    for x in topics: 
        out[x[0]] = x[1];
    
    return np.asarray(out); 

i=0
topic_output(ldamodel,corpus[i],14)

array([0.        , 0.        , 0.        , 0.63974237, 0.        ,
       0.13033921, 0.        , 0.        , 0.        , 0.1358977 ,
       0.        , 0.        , 0.08025853, 0.        ])

In [129]:
print(len(data_df), len(corpus), len(dictionary))

49207 49207 81701


In [130]:
topic_dist = []
for i in range(0,len(data_df)):
    #topic_output(ldamodel,corpus[i])
    #print(i)
    temper = topic_output(ldamodel,corpus[i],14)
    topic_dist.append(temper)

IndexError: index 80397 is out of bounds for axis 1 with size 80397

In [ ]:
topic_dist = np.asarray(topic_dist)
print(topic_dist)

In [ ]:
from scipy.spatial.distance import cdist

val_out = cdist(topic_dist[0].reshape(1,-1), topic_dist, 'cosine')[0]
sorted_ind = np.argsort(val_out)
print(sorted_ind, val_out[363])

In [ ]:
# find nearest neighbors of query document. 
def cosine_neighbors(query_id,topic_dist,kneigh=10):
    val_out = cdist(topic_dist[query_id].reshape(1,-1), 
                    topic_dist, 
                    'cosine')[0]
    val_out = 1 + val_out
    sorted_ind = np.argsort(val_out)
    
    for idx in sorted_ind[0:kneigh]:
        print("document id: ",idx)
        print("ArXiv id: ", data_df.iloc[idx]["id"])
        print("Authors: ", data_df.iloc[idx]["authors"])
        print("Title: ", data_df.iloc[idx]["title"])
        print(data_df.iloc[idx]["abstitle"], " \n")
        print(data_df.iloc[idx]["cleaned_abstitle"], " \n")
        print("topic distribution: \n", topic_dist[idx])
        print("================================================================== \n")

    return sorted_ind[0:kneigh]

In [ ]:
cosine_neighbors(10010,topic_dist,10); 

In [ ]:
for i in range(0,14):
    print(i," --> ", ldamodel.print_topic(i,topn=4))

In [ ]:
ldamodel.load

## Check a famous paper

In [ ]:
data_df[data_df["id"]=="1802.00420"]

In [ ]:
cosine_neighbors(24699,topic_dist,10); 